In [1]:
import numpy as np
import pandas as pd
from itertools import combinations_with_replacement
from scipy.stats import chi2
import scipy.stats as stats
ds = [206,99,103,105,104,489,108]

In [2]:
df = pd.read_excel('df.xlsx')

# Filter to just KHV rows
khv = df[df['Ethnic'] == 'KHV206g'] 

# Filter to just CYP2B6 rows
cyp2d6 = khv[khv['Gene'] == 'CYP2D6']

# Get unique CYP2B6 alleles 
cyp2d6_alleles = set(cyp2d6['Star Allele 1'].unique()) | set(cyp2d6['Star Allele 2'].unique())

print(cyp2d6_alleles)

{32, 1, 2, 35, 4, 36, 71, 10, 45, 14, 17}


In [3]:
cyp2d6_alleles = list(cyp2d6_alleles)
cyp2d6_alleles = [x for x in cyp2d6_alleles if str(x) != 'nan']
cyp2d6_alleles.sort()
'''value_to_delete = 'NaN'
idx = np.where(cyp2d6_alleles == value_to_delete)[0][0]
cyp2d6_alleles = np.delete(cyp2d6_alleles, idx)'''
print(cyp2d6_alleles)


cyp2d6table = pd.DataFrame(cyp2d6_alleles, columns=['CYP2D6'])
cyp2d6table['KHV'] = 0.0
print(cyp2d6table)

[1, 2, 4, 10, 14, 17, 32, 35, 36, 45, 71]
    CYP2D6  KHV
0        1  0.0
1        2  0.0
2        4  0.0
3       10  0.0
4       14  0.0
5       17  0.0
6       32  0.0
7       35  0.0
8       36  0.0
9       45  0.0
10      71  0.0


In [4]:
khv_cyp2d6 = df.query("Ethnic == 'KHV206g' & Gene == 'CYP2D6'")

print(khv_cyp2d6)
start = 0
for allele in cyp2d6_alleles:
  res = 0
  for index, row in khv_cyp2d6.iterrows():
    if allele == row['Star Allele 1']:
      res += row['Percentage per allele']
    if allele == row['Star Allele 2']:  
      res += row['Percentage per allele']
  cyp2d6table.at[start,'KHV'] = res
  start += 1


to_drop = cyp2d6table[cyp2d6table['KHV'] < 1]

# Drop those rows by index
cyp2d6table.drop(index=to_drop.index, inplace=True)
# Divide by 100  
print(cyp2d6table)

      Ethnic              Alleles  Percentage    Gene  Star Allele 1  \
424  KHV206g    CYP2D6*1/CYP2D6*1      9.7090  CYP2D6              1   
425  KHV206g   CYP2D6*1/CYP2D6*10     21.8400  CYP2D6              1   
426  KHV206g   CYP2D6*1/CYP2D6*14      0.4854  CYP2D6              1   
427  KHV206g   CYP2D6*1/CYP2D6*17      0.9709  CYP2D6              1   
428  KHV206g    CYP2D6*1/CYP2D6*2      4.8540  CYP2D6              1   
429  KHV206g   CYP2D6*1/CYP2D6*32      1.4560  CYP2D6              1   
430  KHV206g   CYP2D6*1/CYP2D6*45      0.4854  CYP2D6              1   
431  KHV206g  CYP2D6*10/CYP2D6*10     33.0100  CYP2D6             10   
432  KHV206g  CYP2D6*10/CYP2D6*14      1.9420  CYP2D6             10   
433  KHV206g  CYP2D6*10/CYP2D6*32      4.3690  CYP2D6             10   
434  KHV206g  CYP2D6*10/CYP2D6*36      0.4854  CYP2D6             10   
435  KHV206g  CYP2D6*10/CYP2D6*71      0.9709  CYP2D6             10   
436  KHV206g  CYP2D6*17/CYP2D6*17      0.9709  CYP2D6           

In [5]:
temp = cyp2d6table['CYP2D6'].values.tolist()

ethnics = df['Ethnic'].unique()
ethnics =np.delete(ethnics, -1)
value_to_delete = 'KHV'
idx = np.where(ethnics == value_to_delete)[0][0]
ethnics = np.delete(ethnics, idx)
print(ethnics)


cyp2d6_rows = df[df['Gene'] == 'CYP2D6']

# Create bool series to check if allele in temp
allele1_matches = cyp2d6_rows['Star Allele 1'].isin(temp)
allele2_matches = cyp2d6_rows['Star Allele 2'].isin(temp)

# Filter for rows where either allele matches 
mask = allele1_matches | allele2_matches
cyp2d6_population = cyp2d6_rows[mask]
cyp2d6_population = cyp2d6_population[cyp2d6_population['Ethnic'] != 'KHV206g']

print(cyp2d6_population)#print(temp)

['CEU' 'CHB' 'CHS' 'JPT' 'SAS' 'YRI']
    Ethnic              Alleles  Percentage    Gene  Star Allele 1  \
188    CEU    CYP2D6*1/CYP2D6*1     22.2200  CYP2D6              1   
189    CEU   CYP2D6*1/CYP2D6*10      2.0200  CYP2D6              1   
190    CEU    CYP2D6*1/CYP2D6*2      7.0710  CYP2D6              1   
191    CEU   CYP2D6*1/CYP2D6*28      1.0100  CYP2D6              1   
192    CEU   CYP2D6*1/CYP2D6*32      7.0710  CYP2D6              1   
..     ...                  ...         ...     ...            ...   
320    YRI    CYP2D6*2/CYP2D6*2      4.6300  CYP2D6              2   
321    YRI   CYP2D6*2/CYP2D6*29      2.7780  CYP2D6              2   
322    YRI   CYP2D6*2/CYP2D6*32      0.9259  CYP2D6              2   
323    YRI   CYP2D6*2/CYP2D6*45      0.9259  CYP2D6              2   
328    YRI  CYP2D6*32/CYP2D6*45      0.9259  CYP2D6             32   

     Star Allele 2  Percentage per allele  
188              1               11.11000  
189             10               

In [6]:
for ethnic in ethnics:
        cyp2d6table[ethnic] = 0.0

cyp2d6table = cyp2d6table.reset_index(drop=True)

print(cyp2d6table)

   CYP2D6       KHV  CEU  CHB  CHS  JPT  SAS  YRI
0       1  24.75485  0.0  0.0  0.0  0.0  0.0  0.0
1       2  10.19415  0.0  0.0  0.0  0.0  0.0  0.0
2      10  51.45465  0.0  0.0  0.0  0.0  0.0  0.0
3      14   1.21370  0.0  0.0  0.0  0.0  0.0  0.0
4      17   2.18450  0.0  0.0  0.0  0.0  0.0  0.0
5      32   3.15520  0.0  0.0  0.0  0.0  0.0  0.0


In [7]:
start = 0
for ethnic in ethnics:
    start = 0
    for starallele in temp:
        res = 0

        for index, row in cyp2d6_population.iterrows():
            if row['Ethnic'] == ethnic:
                if starallele == row['Star Allele 1']:
                    res += row['Percentage per allele']
                if starallele == row['Star Allele 2']:  
                    res += row['Percentage per allele']
                cyp2d6table.at[start,ethnic] = res
        #print(start)    
        start += 1
    cyp2d6table[ethnic] = cyp2d6table[ethnic].round(1)

cyp2d6table['KHV'] = cyp2d6table['KHV'].round(1)
cyp2d6table.to_csv('frq/cyp2d6.csv')
print(cyp2d6table)

   CYP2D6   KHV   CEU   CHB   CHS   JPT   SAS   YRI
0       1  24.8  39.9  21.4  22.9  50.0  40.5  26.4
1       2  10.2  13.6  12.1   8.6  13.0  21.8  13.9
2      10  51.5   1.5  56.8  59.5  36.1   5.0   4.6
3      14   1.2   0.0   0.5   0.0   0.5   0.0   0.0
4      17   2.2   0.0   0.0   0.0   0.0   0.0  24.1
5      32   3.2  11.6   3.4   4.8   0.5  11.5   0.9


In [8]:
init = {'CEU': [0], 'CHB': [0], 'CHS': [0], 'JPT': [0], 'SAS': [0], 'YRI': [0]}
gst = pd.DataFrame(init, dtype=float)

h_s_k = 1.0
for index,value in cyp2d6table.iterrows():
    h_s_k -= (cyp2d6table.at[index,'KHV']/100) * (cyp2d6table.at[index,'KHV']/100)

for idx in range(0,6):
    value = 0
    h_t = 1.0
    h_s = 1.0
    for id in range(0,6):
        h_t -= ((cyp2d6table.iloc[id,idx + 2] + cyp2d6table.iloc[id,1])/200) * ((cyp2d6table.iloc[id,idx + 2] + cyp2d6table.iloc[id,1])/200)
        h_s -= (cyp2d6table.iloc[id,idx + 2]/100) * (cyp2d6table.iloc[id,idx + 2]/100)
    h_s = (h_s + h_s_k)/2
    gst.iloc[0,idx] = (h_t - h_s)/h_t # You can modify the value here'''

gst = gst.round(4)
gst.index = ['CYP2D6']
gst.to_csv('gst/gst_cyp2d6.csv')
print(gst)

           CEU     CHB     CHS     JPT     SAS    YRI
CYP2D6  0.0874  0.0019  0.0032  0.0341  0.0837  0.082


In [9]:
test = list(cyp2d6table['CYP2D6'])
test = ['CYP2D6*' + str(int(item)) for item in test]

def generate_gene_combinations(alleles):
    res = []
    length = len(alleles)
    for x in range(0,length):
        for y in range(x,length):
            lol = f'{alleles[x]}/{alleles[y]}'
            res.append(lol)

    return res

print(generate_gene_combinations(test))
gene_combinations = generate_gene_combinations(test)

degrees_of_freedom = len(gene_combinations) - 1
print(gene_combinations)

['CYP2D6*1/CYP2D6*1', 'CYP2D6*1/CYP2D6*2', 'CYP2D6*1/CYP2D6*10', 'CYP2D6*1/CYP2D6*14', 'CYP2D6*1/CYP2D6*17', 'CYP2D6*1/CYP2D6*32', 'CYP2D6*2/CYP2D6*2', 'CYP2D6*2/CYP2D6*10', 'CYP2D6*2/CYP2D6*14', 'CYP2D6*2/CYP2D6*17', 'CYP2D6*2/CYP2D6*32', 'CYP2D6*10/CYP2D6*10', 'CYP2D6*10/CYP2D6*14', 'CYP2D6*10/CYP2D6*17', 'CYP2D6*10/CYP2D6*32', 'CYP2D6*14/CYP2D6*14', 'CYP2D6*14/CYP2D6*17', 'CYP2D6*14/CYP2D6*32', 'CYP2D6*17/CYP2D6*17', 'CYP2D6*17/CYP2D6*32', 'CYP2D6*32/CYP2D6*32']
['CYP2D6*1/CYP2D6*1', 'CYP2D6*1/CYP2D6*2', 'CYP2D6*1/CYP2D6*10', 'CYP2D6*1/CYP2D6*14', 'CYP2D6*1/CYP2D6*17', 'CYP2D6*1/CYP2D6*32', 'CYP2D6*2/CYP2D6*2', 'CYP2D6*2/CYP2D6*10', 'CYP2D6*2/CYP2D6*14', 'CYP2D6*2/CYP2D6*17', 'CYP2D6*2/CYP2D6*32', 'CYP2D6*10/CYP2D6*10', 'CYP2D6*10/CYP2D6*14', 'CYP2D6*10/CYP2D6*17', 'CYP2D6*10/CYP2D6*32', 'CYP2D6*14/CYP2D6*14', 'CYP2D6*14/CYP2D6*17', 'CYP2D6*14/CYP2D6*32', 'CYP2D6*17/CYP2D6*17', 'CYP2D6*17/CYP2D6*32', 'CYP2D6*32/CYP2D6*32']


In [10]:
observed = pd.DataFrame(gene_combinations, columns=['Gene'])
new_columns = ['KHV', 'CEU', 'CHB', 'CHS', 'JPT', 'SAS', 'YRI']
for column in new_columns:
    observed[column] = 0.0

print(observed)

                   Gene  KHV  CEU  CHB  CHS  JPT  SAS  YRI
0     CYP2D6*1/CYP2D6*1  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1     CYP2D6*1/CYP2D6*2  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2    CYP2D6*1/CYP2D6*10  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3    CYP2D6*1/CYP2D6*14  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4    CYP2D6*1/CYP2D6*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5    CYP2D6*1/CYP2D6*32  0.0  0.0  0.0  0.0  0.0  0.0  0.0
6     CYP2D6*2/CYP2D6*2  0.0  0.0  0.0  0.0  0.0  0.0  0.0
7    CYP2D6*2/CYP2D6*10  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8    CYP2D6*2/CYP2D6*14  0.0  0.0  0.0  0.0  0.0  0.0  0.0
9    CYP2D6*2/CYP2D6*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0
10   CYP2D6*2/CYP2D6*32  0.0  0.0  0.0  0.0  0.0  0.0  0.0
11  CYP2D6*10/CYP2D6*10  0.0  0.0  0.0  0.0  0.0  0.0  0.0
12  CYP2D6*10/CYP2D6*14  0.0  0.0  0.0  0.0  0.0  0.0  0.0
13  CYP2D6*10/CYP2D6*17  0.0  0.0  0.0  0.0  0.0  0.0  0.0
14  CYP2D6*10/CYP2D6*32  0.0  0.0  0.0  0.0  0.0  0.0  0.0
15  CYP2D6*14/CYP2D6*14  0.0  0.0  0.0  0.0  0.0  0.0  0

In [11]:
l = len(khv_cyp2d6)
for x in range(0,l):
    if khv_cyp2d6.iloc[x,1] in gene_combinations:
        gene_to_update = khv_cyp2d6.iloc[x,1]
        new_value = khv_cyp2d6.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, 'KHV'] = new_value

length = len(cyp2d6_population)
for x in range(0,length):
    if cyp2d6_population.iloc[x,1] in gene_combinations:
        gene_to_update = cyp2d6_population.iloc[x,1]
        pop_to_update = cyp2d6_population.iloc[x,0]
        new_value = cyp2d6_population.iloc[x,2]
        observed.loc[observed['Gene'] == gene_to_update, pop_to_update ] = new_value 

for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        observed.iloc[y,x+1] = observed.iloc[y,x+1] * ((ds[x] * 1.0)/ 100)

observed = observed.round(0)
print(observed)

                   Gene   KHV   CEU   CHB   CHS   JPT   SAS   YRI
0     CYP2D6*1/CYP2D6*1   6.0  22.0   8.0   7.0  27.0  98.0  12.0
1     CYP2D6*1/CYP2D6*2   4.0   7.0   5.0   3.0  11.0  77.0   5.0
2    CYP2D6*1/CYP2D6*10  56.0   2.0  20.0  26.0  37.0  22.0   3.0
3    CYP2D6*1/CYP2D6*14   2.0   0.0   1.0   0.0   1.0   0.0   0.0
4    CYP2D6*1/CYP2D6*17   2.0   0.0   0.0   0.0   0.0   0.0  13.0
5    CYP2D6*1/CYP2D6*32   2.0   7.0   2.0   4.0   1.0  40.0   0.0
6     CYP2D6*2/CYP2D6*2   4.0   3.0   1.0   2.0   4.0  30.0   5.0
7    CYP2D6*2/CYP2D6*10  21.0   1.0  18.0   8.0   8.0  13.0   2.0
8    CYP2D6*2/CYP2D6*14   0.0   0.0   0.0   0.0   0.0   0.0   0.0
9    CYP2D6*2/CYP2D6*17   2.0   0.0   0.0   0.0   0.0   0.0   8.0
10   CYP2D6*2/CYP2D6*32   0.0   4.0   0.0   2.0   0.0  30.0   1.0
11  CYP2D6*10/CYP2D6*10  81.0   0.0  35.0  43.0  15.0   2.0   1.0
12  CYP2D6*10/CYP2D6*14   8.0   0.0   0.0   0.0   0.0   0.0   0.0
13  CYP2D6*10/CYP2D6*17   0.0   0.0   0.0   0.0   0.0   0.0   2.0
14  CYP2D6

In [12]:
hypo = pd.DataFrame(gene_combinations, columns=['Gene'])
for column in new_columns:
    hypo[column] = 0.0

allele = cyp2d6table.iloc[:,0].tolist()

for x in range(0,7):
    temp = cyp2d6table.iloc[:,x + 1].tolist()
    for allele1 in range(0,len(temp)):
        for allele2 in range(allele1,len(temp)):
            if allele1 == allele2:
                new_value = (temp[allele1] / 100) * (temp[allele2] / 100)
            elif allele1 != allele2:
                new_value = 2 * (temp[allele1] / 100) * (temp[allele2] / 100)
            gene = f"CYP2D6*{int(allele[allele1])}/CYP2D6*{int(allele[allele2])}"
            
            row_index = hypo.index[hypo['Gene'] == gene].tolist()[0]
            hypo.iloc[row_index, x+1] = new_value

print(hypo)

                   Gene       KHV       CEU       CHB       CHS       JPT  \
0     CYP2D6*1/CYP2D6*1  0.061504  0.159201  0.045796  0.052441  0.250000   
1     CYP2D6*1/CYP2D6*2  0.050592  0.108528  0.051788  0.039388  0.130000   
2    CYP2D6*1/CYP2D6*10  0.255440  0.011970  0.243104  0.272510  0.361000   
3    CYP2D6*1/CYP2D6*14  0.005952  0.000000  0.002140  0.000000  0.005000   
4    CYP2D6*1/CYP2D6*17  0.010912  0.000000  0.000000  0.000000  0.000000   
5    CYP2D6*1/CYP2D6*32  0.015872  0.092568  0.014552  0.021984  0.005000   
6     CYP2D6*2/CYP2D6*2  0.010404  0.018496  0.014641  0.007396  0.016900   
7    CYP2D6*2/CYP2D6*10  0.105060  0.004080  0.137456  0.102340  0.093860   
8    CYP2D6*2/CYP2D6*14  0.002448  0.000000  0.001210  0.000000  0.001300   
9    CYP2D6*2/CYP2D6*17  0.004488  0.000000  0.000000  0.000000  0.000000   
10   CYP2D6*2/CYP2D6*32  0.006528  0.031552  0.008228  0.008256  0.001300   
11  CYP2D6*10/CYP2D6*10  0.265225  0.000225  0.322624  0.354025  0.130321   

In [13]:
for x in range(0,7):
    for y in range(0,len(gene_combinations)):
        hypo.iloc[y,x+1] = hypo.iloc[y,x+1] * ((ds[x] * 1.0))
        if hypo.iloc[y,x+1] < 1:
            hypo.iloc[y,x+1] = 1
hypo = hypo.round(0)
print(hypo)

                   Gene   KHV   CEU   CHB   CHS   JPT   SAS   YRI
0     CYP2D6*1/CYP2D6*1  13.0  16.0   5.0   6.0  26.0  80.0   8.0
1     CYP2D6*1/CYP2D6*2  10.0  11.0   5.0   4.0  14.0  86.0   8.0
2    CYP2D6*1/CYP2D6*10  53.0   1.0  25.0  29.0  38.0  20.0   3.0
3    CYP2D6*1/CYP2D6*14   1.0   1.0   1.0   1.0   1.0   1.0   1.0
4    CYP2D6*1/CYP2D6*17   2.0   1.0   1.0   1.0   1.0   1.0  14.0
5    CYP2D6*1/CYP2D6*32   3.0   9.0   1.0   2.0   1.0  46.0   1.0
6     CYP2D6*2/CYP2D6*2   2.0   2.0   2.0   1.0   2.0  23.0   2.0
7    CYP2D6*2/CYP2D6*10  22.0   1.0  14.0  11.0  10.0  11.0   1.0
8    CYP2D6*2/CYP2D6*14   1.0   1.0   1.0   1.0   1.0   1.0   1.0
9    CYP2D6*2/CYP2D6*17   1.0   1.0   1.0   1.0   1.0   1.0   7.0
10   CYP2D6*2/CYP2D6*32   1.0   3.0   1.0   1.0   1.0  25.0   1.0
11  CYP2D6*10/CYP2D6*10  55.0   1.0  33.0  37.0  14.0   1.0   1.0
12  CYP2D6*10/CYP2D6*14   3.0   1.0   1.0   1.0   1.0   1.0   1.0
13  CYP2D6*10/CYP2D6*17   5.0   1.0   1.0   1.0   1.0   1.0   2.0
14  CYP2D6

In [14]:
hihi = {"KHV": [0.0], "CEU": [0.0], "CHB": [0.0], "CHS": [0.0], "JPT": [0.0], "SAS": [0.0], "YRI": [0.0]}
chi_square = pd.DataFrame(hihi)
print(chi_square)
for x in range(0,7):
    res = 0
    for y in range(0,len(hypo)):
        chi_square_value = ((hypo.iloc[y,x+1] - observed.iloc[y,x+1]) * (hypo.iloc[y,x+1] - observed.iloc[y,x+1])) / (hypo.iloc[y,x+1])
        res += chi_square_value

    p_value = 1 - stats.chi2.cdf(res, degrees_of_freedom)
    chi_square.iloc[0,x] = p_value 

chi_square = chi_square.round(4)

chi_square.index = ['CYP2D6']
print(chi_square)

   KHV  CEU  CHB  CHS  JPT  SAS  YRI
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
           KHV    CEU     CHB     CHS     JPT     SAS     YRI
CYP2D6  0.0009  0.459  0.5997  0.5098  0.7055  0.1725  0.3679


In [ ]:
print(cyp2d6table)
def f(row):
    return "CYP2D6*" + str(row)
cyp2d6table["CYP2D6"] = cyp2d6table["CYP2D6"].apply(f)
cyp2d6table.rename(columns={"CYP2D6": "Gene"}, inplace=True)
cyp2d6table.to_excel("heatmap/cyp2d6table.xlsx")